In [4]:
!gdown 1ocbCB3PnBIlzYMHc6wnEo_Vzm1XYwlMk

Downloading...
From: https://drive.google.com/uc?id=1ocbCB3PnBIlzYMHc6wnEo_Vzm1XYwlMk
To: /content/penjualan_new.db
100% 135k/135k [00:00<00:00, 90.3MB/s]


In [5]:
# Install necessary packages
!pip install -q ipywidgets
!pip install -q qrcode

import sqlite3
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import qrcode
from datetime import datetime
import random
import io
import base64
from PIL import Image
import re

# Global variables
current_user = None
user_type = None
conn = None
cursor = None

# Blue theme styles
BLUE_STYLE = """
<style>
.blue-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    text-align: center;
    margin-bottom: 20px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}
.blue-card {
    background: white;
    border: 2px solid #667eea;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    box-shadow: 0 2px 4px rgba(102,126,234,0.1);
}
.info-text {
    color: #4a5568;
    font-size: 14px;
}
.success-msg {
    color: #38a169;
    background: #f0fff4;
    padding: 10px;
    border-radius: 5px;
    border-left: 4px solid #38a169;
}
.error-msg {
    color: #e53e3e;
    background: #fff5f5;
    padding: 10px;
    border-radius: 5px;
    border-left: 4px solid #e53e3e;
}
</style>
"""

# Initialize database connection
def init_db():
    global conn, cursor
    conn = sqlite3.connect('/content/penjualan_new.db')
    cursor = conn.cursor()

# Function to create styled button
def create_button(desc, style='primary', width='200px'):
    colors = {
        'primary': '#667eea',
        'success': '#48bb78',
        'danger': '#f56565',
        'secondary': '#718096'
    }

    button = widgets.Button(description=desc)
    button.style.button_color = colors.get(style, colors['primary'])
    button.layout.width = width
    button.layout.margin = '5px'
    return button

# Function to create styled input
def create_input(desc, placeholder='', input_type='text'):
    if input_type == 'password':
        widget = widgets.Password(description=desc, placeholder=placeholder)
    elif input_type == 'textarea':
        widget = widgets.Textarea(description=desc, placeholder=placeholder)
    elif input_type == 'int':
        widget = widgets.IntText(description=desc, value=1, min=1)
    elif input_type == 'float':
        widget = widgets.FloatText(description=desc, value=0.0, min=0.0)
    else:
        widget = widgets.Text(description=desc, placeholder=placeholder)

    widget.layout.width = '300px'
    widget.layout.margin = '5px'
    return widget

# Function to convert an image to base64 for display in HTML
def image_to_base64(img):
    buffered = io.BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return img_str

# Function to generate a QR code
def generate_qr(data):
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=4)
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    return img

# Login functions
def validate_login(email, password, user_role):
    global current_user, user_type
    if user_role == "admin":
        cursor.execute("SELECT * FROM admin WHERE email = ? AND password = ?", (email, password))
        user = cursor.fetchone()
        if user:
            current_user = email
            user_type = "admin"
            return True
    else:
        cursor.execute("SELECT * FROM customers WHERE email = ? AND password = ?", (email, password))
        user = cursor.fetchone()
        if user:
            current_user = user[0]
            user_type = "customer"
            return True
    return False

# Register functions
def register_customer(name, email, address, phone, password):
    try:
        cursor.execute("SELECT * FROM customers WHERE email = ?", (email,))
        if cursor.fetchone():
            return False, "Email sudah terdaftar"

        def generate_customer_id():
            cursor.execute("SELECT customer_id FROM customers")
            all_ids = cursor.fetchall()
            max_num = 0
            for row in all_ids:
                try:
                    num = int(row[0].replace("CUST", ""))
                    if num > max_num:
                        max_num = num
                except:
                    continue
            return f"CUST{max_num + 1:03d}"

        new_id = generate_customer_id()
        cursor.execute("""
            INSERT INTO customers (customer_id, name_cust, email, address, phone_number, password)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (new_id, name, email, address, phone, password))
        conn.commit()
        return True, "Registrasi berhasil!"
    except Exception as e:
        conn.rollback()
        return False, f"Registrasi gagal: {str(e)}"

# Product functions
def get_all_products():
    cursor.execute("SELECT * FROM product")
    return cursor.fetchall()

def search_products(query):
    cursor.execute("SELECT * FROM product WHERE product_name LIKE ? OR category LIKE ?",
                  (f"%{query}%", f"%{query}%"))
    return cursor.fetchall()

# Order functions
def generate_order_id():
    cursor.execute("SELECT order_id FROM Orders")
    all_ids = cursor.fetchall()
    max_num = 0
    for row in all_ids:
        try:
            num = int(row[0].replace("ORD", ""))
            if num > max_num:
                max_num = num
        except:
            continue
    return f"ORD{max_num + 1:05d}"

def create_order(user, cart_items, payment_method):
    """
    cart_items: list of tuples [(product_id, quantity), ...]
    """
    try:
        # Validate all products and check stock
        total_price = 0
        validated_items = []

        for product_id, quantity in cart_items:
            cursor.execute("SELECT product_id, product_name, price, stock FROM product WHERE product_id = ?", (product_id,))
            product = cursor.fetchone()

            if not product:
                return False, f"Produk {product_id} tidak ditemukan"
            if product[3] < quantity:
                return False, f"Stok produk {product[1]} tidak mencukupi (tersedia: {product[3]})"

            # Convert price from thousands to actual rupiah
            price_per_unit = product[2] * 1000
            item_total = price_per_unit * quantity
            total_price += item_total

            validated_items.append((product_id, product[1], quantity, price_per_unit, item_total))

        # Create order
        order_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        order_id = generate_order_id()

        cursor.execute("INSERT INTO Orders (order_id, customer_id, order_date, status) VALUES (?, ?, ?, ?)",
                      (order_id, user, order_date, "pending"))

        # Insert order details for each item
        for product_id, product_name, quantity, price_per_unit, item_total in validated_items:
            cursor.execute("INSERT INTO Order_Details (order_id, product_id, quantity, price_total) VALUES (?, ?, ?, ?)",
                          (order_id, product_id, quantity, item_total / 1000))  # Store in thousands

            # Update stock
            cursor.execute("UPDATE product SET stock = stock - ? WHERE product_id = ?", (quantity, product_id))

        # Insert payment record
        cursor.execute("INSERT INTO Order_Payments (order_id, payment_type, payment_status, amount_paid) VALUES (?, ?, ?, ?)",
                      (order_id, payment_method, "pending", total_price / 1000))  # Store in thousands

        conn.commit()
        return True, order_id
    except Exception as e:
        conn.rollback()
        return False, f"Error: {str(e)}"

# UI Functions
def show_login_page():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>🛒 BRWSC Hardware Store</h1><p>Silakan login untuk melanjutkan</p></div>'))

    email_input = create_input("Email", "Masukkan email Anda")
    password_input = create_input("Password", "Masukkan password", "password")
    role_dropdown = widgets.Dropdown(
        options=[('Customer', 'customer'), ('Admin', 'admin')],
        value='customer', description='Login as:'
    )
    role_dropdown.layout.width = '300px'

    login_btn = create_button("🔑 Login", "primary")
    register_btn = create_button("📝 Daftar Sebagai Customer", "success")
    message_output = widgets.Output()

    def on_login_click(b):
        with message_output:
            clear_output()
            if not email_input.value or not password_input.value:
                display(HTML('<div class="error-msg">Mohon isi email dan password</div>'))
                return

            if validate_login(email_input.value, password_input.value, role_dropdown.value):
                display(HTML('<div class="success-msg">Login berhasil! Mengalihkan...</div>'))
                if user_type == "admin":
                    show_admin_main_menu()
                else:
                    show_customer_main_menu()
            else:
                display(HTML('<div class="error-msg">Email atau password salah</div>'))

    def on_register_click(b):
        show_register_page()

    login_btn.on_click(on_login_click)
    register_btn.on_click(on_register_click)

    form_box = widgets.VBox([
        email_input, password_input, role_dropdown,
        widgets.HBox([login_btn, register_btn], layout=widgets.Layout(justify_content='center'))
    ], layout=widgets.Layout(align_items='center'))

    display(form_box)
    display(message_output)

def show_register_page():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>📝 Daftar Akun Baru</h1></div>'))

    name_input = create_input("Nama", "Nama lengkap")
    email_input = create_input("Email", "Email aktif")
    address_input = create_input("Alamat", "Alamat lengkap", "textarea")
    phone_input = create_input("Telepon", "Nomor telepon")
    password_input = create_input("Password", "Buat password", "password")
    confirm_password = create_input("Konfirmasi", "Ulangi password", "password")

    register_btn = create_button("✅ Daftar", "success")
    back_btn = create_button("← Kembali", "secondary")
    message_output = widgets.Output()

    def on_register_click(b):
        with message_output:
            clear_output()
            if not all([name_input.value, email_input.value, address_input.value, phone_input.value, password_input.value]):
                display(HTML('<div class="error-msg">Mohon isi semua field</div>'))
                return

            if password_input.value != confirm_password.value:
                display(HTML('<div class="error-msg">Password tidak cocok</div>'))
                return

            success, msg = register_customer(name_input.value, email_input.value, address_input.value, phone_input.value, password_input.value)

            if success:
                display(HTML(f'<div class="success-msg">{msg}</div>'))
            else:
                display(HTML(f'<div class="error-msg">{msg}</div>'))

    def on_back_click(b):
        show_login_page()

    register_btn.on_click(on_register_click)
    back_btn.on_click(on_back_click)

    form_box = widgets.VBox([
        name_input, email_input, address_input, phone_input, password_input, confirm_password,
        widgets.HBox([register_btn, back_btn], layout=widgets.Layout(justify_content='center'))
    ], layout=widgets.Layout(align_items='center'))

    display(form_box)
    display(message_output)

def show_customer_main_menu():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>🏠 Menu Utama Customer</h1></div>'))

    profile_btn = create_button("👤 Profil Saya", "primary", "250px")
    search_btn = create_button("🔍 Cari Produk", "primary", "250px")
    order_btn = create_button("🛒 Buat Pesanan", "success", "250px")
    logout_btn = create_button("🚪 Logout", "danger", "250px")

    def on_profile_click(b): show_customer_profile()
    def on_search_click(b): show_product_search()
    def on_order_click(b): show_order_page()
    def on_logout_click(b):
        global current_user, user_type
        current_user = None
        user_type = None
        show_login_page()

    profile_btn.on_click(on_profile_click)
    search_btn.on_click(on_search_click)
    order_btn.on_click(on_order_click)
    logout_btn.on_click(on_logout_click)

    menu_box = widgets.VBox([profile_btn, search_btn, order_btn, logout_btn],
                           layout=widgets.Layout(align_items='center'))
    display(menu_box)

def show_admin_product_management():
    clear_output()

    # Create title with enhanced styling
    title = widgets.HTML("""
        <div style="
            text-align: center;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border-radius: 15px;
            margin-bottom: 25px;
            box-shadow: 0 8px 16px rgba(102, 126, 234, 0.3);
        ">
            <h1 style="margin: 0; font-size: 28px; font-weight: 300; letter-spacing: 1px;">
                📦 Product Management
            </h1>
            <p style="margin: 8px 0 0 0; opacity: 0.9; font-size: 14px;">
                Manage your products efficiently
            </p>
        </div>
    """)

    # Enhanced button styling
    button_style = {
        'button_color': '#4A90E2',
        'font_weight': '500',
        'font_size': '14px'
    }

    button_layout = widgets.Layout(
        width='200px',
        height='50px',
        margin='8px',
        border_radius='8px'
    )

    # Create buttons with icons and enhanced styling
    add_button = widgets.Button(
        description="➕ Add New Product",
        style=button_style,
        layout=button_layout,
        tooltip="Add a new product to inventory"
    )

    delete_button = widgets.Button(
        description="🗑️ Delete Product",
        style=dict(button_style, button_color='#E74C3C'),
        layout=button_layout,
        tooltip="Remove product from inventory"
    )

    view_button = widgets.Button(
        description="👁️ View All Products",
        style=dict(button_style, button_color='#3498DB'),
        layout=button_layout,
        tooltip="Browse all products"
    )

    back_button = widgets.Button(
        description="🏠 Back to Main Menu",
        style=dict(button_style, button_color='#95A5A6'),
        layout=button_layout,
        tooltip="Return to main menu"
    )

    # Event handlers (unchanged)
    def on_add_click(b):
        show_admin_add_product()

    def on_delete_click(b):
        show_admin_delete_product()

    def on_view_click(b):
        show_admin_product_search()

    def on_back_click(b):
        show_admin_main_menu()

    # Attach event handlers
    add_button.on_click(on_add_click)
    delete_button.on_click(on_delete_click)
    view_button.on_click(on_view_click)
    back_button.on_click(on_back_click)

    # Create button container with nice layout
    button_container = widgets.VBox([
        widgets.HBox([add_button, view_button], layout=widgets.Layout(justify_content='center')),
        widgets.HBox([delete_button, back_button], layout=widgets.Layout(justify_content='center'))
    ], layout=widgets.Layout(
        align_items='center',
        padding='20px',
        background_color='#f8f9fa',
        border_radius='12px',
        border='1px solid #e9ecef'
    ))

    # Display all widgets
    display(title)
    display(button_container)

def show_admin_main_menu():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>⚙️ Menu Admin</h1></div>'))

    product_btn = create_button("📦 Kelola Produk", "primary", "250px")
    search_btn = create_button("🔍 Lihat Produk", "primary", "250px")
    orders_btn = create_button("📋 Database Order", "primary", "250px")
    logout_btn = create_button("🚪 Logout", "danger", "250px")

    def on_product_click(b): show_admin_product_management()
    def on_search_click(b): show_admin_product_search()
    def on_orders_click(b): show_admin_order_database()
    def on_logout_click(b):
        global current_user, user_type
        current_user = None
        user_type = None
        show_login_page()

    product_btn.on_click(on_product_click)
    search_btn.on_click(on_search_click)
    orders_btn.on_click(on_orders_click)
    logout_btn.on_click(on_logout_click)

    menu_box = widgets.VBox([product_btn, search_btn, orders_btn, logout_btn],
                           layout=widgets.Layout(align_items='center'))
    display(menu_box)

def show_product_search():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>🔍 Cari Produk</h1></div>'))

    search_input = create_input("", "Masukkan nama produk atau kategori")
    search_btn = create_button("🔍 Cari", "primary")
    back_btn = create_button("← Kembali", "secondary")
    results_output = widgets.Output()

    def on_search_click(b):
        with results_output:
            clear_output()
            if search_input.value:
                products = search_products(search_input.value)
            else:
                products = get_all_products()

            if products:
                # Convert prices to actual rupiah for display
                products_display = []
                for p in products:
                    product_list = list(p)
                    product_list[3] = f"Rp {p[3] * 1000:,.0f}"  # Convert price
                    products_display.append(product_list)

                df = pd.DataFrame(products_display, columns=['ID', 'Nama', 'Kategori', 'Harga', 'Deskripsi', 'Stok', 'Rating'])
                display(HTML(f'<div class="blue-card">{df.to_html(index=False)}</div>'))
            else:
                display(HTML('<div class="error-msg">Tidak ada produk ditemukan</div>'))

    def on_back_click(b):
        show_customer_main_menu()

    search_btn.on_click(on_search_click)
    back_btn.on_click(on_back_click)

    controls = widgets.HBox([search_input, search_btn], layout=widgets.Layout(justify_content='center'))
    display(controls)
    display(widgets.HBox([back_btn], layout=widgets.Layout(justify_content='center')))
    display(results_output)

    # Show all products initially with correct prices
    with results_output:
        products = get_all_products()
        if products:
            products_display = []
            for p in products:
                product_list = list(p)
                product_list[3] = f"Rp {p[3] * 1000:,.0f}"
                products_display.append(product_list)

            df = pd.DataFrame(products_display, columns=['ID', 'Nama', 'Kategori', 'Harga', 'Deskripsi', 'Stok', 'Rating'])
            display(HTML(f'<div class="blue-card">{df.to_html(index=False)}</div>'))

def show_order_confirmation(order_id):
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>✅ Konfirmasi Pesanan</h1></div>'))

    cursor.execute("""
        SELECT o.order_id, o.order_date, o.status, od.quantity, od.price_total,
               p.product_name, p.price, op.payment_type
        FROM Orders o
        JOIN Order_Details od ON o.order_id = od.order_id
        JOIN product p ON od.product_id = p.product_id
        JOIN Order_Payments op ON o.order_id = op.order_id
        WHERE o.order_id = ?
    """, (order_id,))

    order_info = cursor.fetchone()

    if order_info:
        order_html = f"""
        <div class="blue-card">
            <h3>🧾 Detail Pesanan</h3>
            <p><strong>ID Pesanan:</strong> {order_info[0]}</p>
            <p><strong>Tanggal:</strong> {order_info[1]}</p>
            <p><strong>Produk:</strong> {order_info[5]}</p>
            <p><strong>Jumlah:</strong> {order_info[3]}</p>
            <p><strong>Harga Satuan:</strong> Rp{order_info[6]:,.0f}</p>
            <p><strong>Total:</strong> Rp{order_info[4]:,.0f}</p>
            <p><strong>Pembayaran:</strong> {order_info[7]}</p>
        </div>
        """
        display(HTML(order_html))

    proceed_btn = create_button("💳 Bayar Sekarang", "success")
    back_btn = create_button("← Kembali", "secondary")

    def on_proceed_click(b):
        show_payment_page(order_id, order_info[7])

    def on_back_click(b):
        show_customer_main_menu()

    proceed_btn.on_click(on_proceed_click)
    back_btn.on_click(on_back_click)

    display(widgets.HBox([proceed_btn, back_btn], layout=widgets.Layout(justify_content='center')))

def show_order_page():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>🛒 Buat Pesanan</h1><p>Tambahkan produk ke keranjang belanja</p></div>'))

    # Cart storage
    cart = []

    # Get all products
    products = get_all_products()
    product_options = []
    for p in products:
        try:
            price = p[3] * 1000
            option_text = f"{p[0]} - {p[1]} (Rp{price:,.0f}) - Stok: {p[5]}"
            product_options.append((option_text, p[0]))
        except (ValueError, TypeError):
            option_text = f"{p[0]} - {p[1]} (Rp{p[3]}) - Stok: {p[5]}"
            product_options.append((option_text, p[0]))

    # Product selection widgets
    product_dropdown = widgets.Dropdown(options=product_options, description='Pilih Produk:')
    product_dropdown.layout.width = '450px'

    quantity_input = create_input("Jumlah", "", "int")
    add_to_cart_btn = create_button("➕ Tambah ke Keranjang", "success")

    # Cart display
    cart_output = widgets.Output()

    # Payment method
    payment_dropdown = widgets.Dropdown(
        options=[('E-Wallet', 'e-wallet'), ('Transfer Bank', 'bank_transfer'),
                ('Kartu Debit', 'debit_card'), ('Kartu Kredit', 'credit_card')],
        description='Pembayaran:'
    )
    payment_dropdown.layout.width = '300px'

    # Order buttons
    checkout_btn = create_button("💳 Checkout", "primary")
    clear_cart_btn = create_button("🗑️ Kosongkan Keranjang", "danger")
    back_btn = create_button("← Kembali", "secondary")

    message_output = widgets.Output()

    def update_cart_display():
        with cart_output:
            clear_output()
            if not cart:
                display(HTML('<div class="blue-card"><h3>🛒 Keranjang Kosong</h3><p>Tambahkan produk ke keranjang untuk melanjutkan</p></div>'))
                return

            total_price = 0
            cart_html = '<div class="blue-card"><h3>🛒 Keranjang Belanja</h3><table style="width:100%; border-collapse: collapse;">'
            cart_html += '<tr style="background:#667eea; color:white;"><th style="padding:10px; border:1px solid #ddd;">Produk</th><th style="padding:10px; border:1px solid #ddd;">Jumlah</th><th style="padding:10px; border:1px solid #ddd;">Harga Satuan</th><th style="padding:10px; border:1px solid #ddd;">Total</th><th style="padding:10px; border:1px solid #ddd;">Aksi</th></tr>'

            for i, (product_id, product_name, quantity, price_per_unit) in enumerate(cart):
                item_total = price_per_unit * quantity
                total_price += item_total
                cart_html += f'<tr style="background:#f8f9fa;"><td style="padding:8px; border:1px solid #ddd;">{product_name}</td><td style="padding:8px; border:1px solid #ddd; text-align:center;">{quantity}</td><td style="padding:8px; border:1px solid #ddd; text-align:right;">Rp{price_per_unit:,.0f}</td><td style="padding:8px; border:1px solid #ddd; text-align:right;">Rp{item_total:,.0f}</td><td style="padding:8px; border:1px solid #ddd; text-align:center;"><button onclick="removeFromCart({i})" style="background:#f56565; color:white; border:none; padding:5px 10px; border-radius:3px; cursor:pointer;">Hapus</button></td></tr>'

            cart_html += f'<tr style="background:#667eea; color:white; font-weight:bold;"><td colspan="3" style="padding:10px; border:1px solid #ddd; text-align:right;">TOTAL:</td><td style="padding:10px; border:1px solid #ddd; text-align:right;">Rp{total_price:,.0f}</td><td style="padding:10px; border:1px solid #ddd;"></td></tr>'
            cart_html += '</table></div>'

            display(HTML(cart_html))

    def on_add_to_cart_click(b):
        if not product_dropdown.value or quantity_input.value <= 0:
            with message_output:
                clear_output()
                display(HTML('<div class="error-msg">Mohon pilih produk dan masukkan jumlah yang valid</div>'))
            return

        # Get product info
        cursor.execute("SELECT product_id, product_name, price, stock FROM product WHERE product_id = ?", (product_dropdown.value,))
        product = cursor.fetchone()

        if not product:
            with message_output:
                clear_output()
                display(HTML('<div class="error-msg">Produk tidak ditemukan</div>'))
            return

        # Check if product already in cart
        existing_qty = 0
        for i, (cart_product_id, _, cart_qty, _) in enumerate(cart):
            if cart_product_id == product_dropdown.value:
                existing_qty = cart_qty
                cart.pop(i)
                break

        total_qty = existing_qty + quantity_input.value

        if total_qty > product[3]:
            with message_output:
                clear_output()
                display(HTML(f'<div class="error-msg">Stok tidak mencukupi. Stok tersedia: {product[3]}, di keranjang: {existing_qty}</div>'))
            return

        # Add to cart
        price_per_unit = product[2] * 1000
        cart.append((product[0], product[1], total_qty, price_per_unit))

        # Reset inputs
        quantity_input.value = 1
        update_cart_display()

        with message_output:
            clear_output()
            display(HTML('<div class="success-msg">Produk berhasil ditambahkan ke keranjang!</div>'))

    def on_checkout_click(b):
        if not cart:
            with message_output:
                clear_output()
                display(HTML('<div class="error-msg">Keranjang kosong</div>'))
            return

        # Convert cart to format expected by create_order
        cart_items = [(product_id, quantity) for product_id, _, quantity, _ in cart]

        success, result = create_order(current_user, cart_items, payment_dropdown.value)

        if success:
            with message_output:
                clear_output()
                display(HTML('<div class="success-msg">Pesanan berhasil dibuat!</div>'))
            show_order_confirmation(result)
        else:
            with message_output:
                clear_output()
                display(HTML(f'<div class="error-msg">{result}</div>'))

    def on_clear_cart_click(b):
        cart.clear()
        update_cart_display()
        with message_output:
            clear_output()
            display(HTML('<div class="success-msg">Keranjang dikosongkan</div>'))

    def on_back_click(b):
        show_customer_main_menu()

    # Event handlers
    add_to_cart_btn.on_click(on_add_to_cart_click)
    checkout_btn.on_click(on_checkout_click)
    clear_cart_btn.on_click(on_clear_cart_click)
    back_btn.on_click(on_back_click)

    # Layout
    product_section = widgets.VBox([
        widgets.HTML('<div class="blue-card"><h3>📦 Pilih Produk</h3></div>'),
        product_dropdown,
        quantity_input,
        add_to_cart_btn
    ], layout=widgets.Layout(align_items='center'))

    checkout_section = widgets.VBox([
        widgets.HTML('<div class="blue-card"><h3>💳 Checkout</h3></div>'),
        payment_dropdown,
        widgets.HBox([checkout_btn, clear_cart_btn], layout=widgets.Layout(justify_content='center')),
        widgets.HBox([back_btn], layout=widgets.Layout(justify_content='center'))
    ], layout=widgets.Layout(align_items='center'))

    display(product_section)
    display(cart_output)
    display(checkout_section)
    display(message_output)

    # Initialize cart display
    update_cart_display()

def show_payment_page(order_id, payment_type):
    clear_output()
    display(HTML(BLUE_STYLE))

    # Blue header
    display(HTML('<div class="blue-header"><h1>💳 Pembayaran</h1></div>'))

    # Different payment interfaces based on payment type
    if payment_type in ["e-wallet", "bank_transfer"]:
        # Generate QR code
        qr_data = f"Order:{order_id}|Amount:{random.randint(100000, 999999)}|Date:{datetime.now().strftime('%Y%m%d%H%M%S')}"
        qr_img = generate_qr(qr_data)
        img_base64 = image_to_base64(qr_img)

        payment_html = f"""
        <div class="blue-card" style="text-align: center;">
            <h3>📱 {payment_type.replace('_', ' ').title()}</h3>
            <p class="info-text">Scan QR code di bawah untuk menyelesaikan pembayaran:</p>
            <img src="data:image/png;base64,{img_base64}" width="200" height="200">
            <p class="info-text">Atau gunakan detail pembayaran berikut:</p>
            <p><strong>Order ID:</strong> {order_id}</p>
            <p><strong>No. Rekening:</strong> 1234-5678-9012-3456</p>
            <p><strong>Bank:</strong> Example Bank</p>
        </div>
        """
        display(HTML(payment_html))
        payment_interface = None

    else:  # Debit or Credit Card
        display(HTML(f"""
        <div class="blue-card">
            <h3>💳 {payment_type.replace('_', ' ').title()}</h3>
            <p class="info-text">Masukkan detail kartu Anda dengan aman</p>
        </div>
        """))

        card_number = widgets.Text(
            description="🔢 Nomor Kartu:",
            placeholder="XXXX-XXXX-XXXX-XXXX",
            layout=widgets.Layout(width='300px', margin='5px')
        )
        expiry_date = widgets.Text(
            description="📅 Expired:",
            placeholder="MM/YY",
            layout=widgets.Layout(width='300px', margin='5px')
        )
        cvv = widgets.Password(
            description="🔒 CVV:",
            placeholder="XXX",
            layout=widgets.Layout(width='300px', margin='5px')
        )

        payment_interface = widgets.VBox([
            card_number,
            expiry_date,
            cvv
        ], layout=widgets.Layout(align_items='center'))

    # Buttons with blue styling
    confirm_button = widgets.Button(
        description="✅ Konfirmasi Pembayaran",
        button_style='primary',
        layout=widgets.Layout(width='180px', margin='10px 5px')
    )
    confirm_button.style.button_color = '#667eea'

    back_button = widgets.Button(
        description="← Kembali",
        button_style='info',
        layout=widgets.Layout(width='120px', margin='10px 5px')
    )
    back_button.style.button_color = '#718096'

    message_output = widgets.Output()

    # Event handlers
    def on_confirm_click(b):
        with message_output:
            clear_output()
            # Check if card payment and validate fields
            if payment_type not in ["e-wallet", "bank_transfer"]:
                if not all([card_number.value, expiry_date.value, cvv.value]):
                    display(HTML('<div class="error-msg">Mohon isi semua field kartu</div>'))
                    return

            # Process payment
            if update_payment_status(order_id):
                display(HTML('<div class="success-msg">✅ Pembayaran berhasil!</div>'))
                confirm_button.disabled = True
            else:
                display(HTML('<div class="error-msg">❌ Pembayaran gagal. Silakan coba lagi.</div>'))

    def on_back_click(b):
        show_customer_main_menu()

    confirm_button.on_click(on_confirm_click)
    back_button.on_click(on_back_click)

    # Display widgets
    if payment_interface:
      display(payment_interface)

    display(widgets.HBox([confirm_button, back_button],
                        layout=widgets.Layout(justify_content='center')))
    display(message_output)

def update_payment_status(order_id):
    try:
        cursor.execute("UPDATE Order_Payments SET payment_status = 'paid' WHERE order_id = ?", (order_id,))
        cursor.execute("UPDATE Orders SET status = 'proceed' WHERE order_id = ?", (order_id,))
        conn.commit()
        return True
    except:
        conn.rollback()
        return False

def show_customer_profile():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>👤 Profil Saya</h1></div>'))

    # Get customer info
    cursor.execute("SELECT * FROM customers WHERE customer_id = ?", (current_user,))
    customer_info = cursor.fetchone()

    if customer_info:
        profile_html = f"""
        <div class="blue-card">
            <h3>ℹ️ Informasi Personal</h3>
            <div style="background: linear-gradient(135deg, #f0f7ff 0%, #e8f4fd 100%); padding: 20px; border-radius: 10px; margin: 15px 0; border: 1px solid #42a5f5;">
                <p style="margin: 10px 0; color: #000;"><strong style="color: #1976d2;">👤 Nama:</strong> {customer_info[1]}</p>
                <p style="margin: 10px 0; color: #000;"><strong style="color: #1976d2;">📧 Email:</strong> {customer_info[2]}</p>
                <p style="margin: 10px 0; color: #000;"><strong style="color: #1976d2;">🏠 Alamat:</strong> {customer_info[3]}</p>
                <p style="margin: 10px 0; color: #000;"><strong style="color: #1976d2;">📞 Telepon:</strong> {customer_info[4]}</p>
            </div>
        </div>
        """
        display(HTML(profile_html))

    # Get order information with details
    cursor.execute("""
        SELECT o.order_id, o.order_date, o.status, p.payment_status, p.amount_paid
        FROM orders o
        LEFT JOIN order_payments p ON o.order_id = p.order_id
        WHERE o.customer_id = ?
        ORDER BY o.order_date DESC
    """, (current_user,))
    orders = cursor.fetchall()

    if orders:
        # Statistics
        unique_orders = list(set([order[0] for order in orders]))
        total_orders = len(unique_orders)

        cursor.execute("""
            SELECT COALESCE(SUM(p.amount_paid), 0) as total
            FROM orders o
            LEFT JOIN order_payments p ON o.order_id = p.order_id
            WHERE o.customer_id = ? AND p.amount_paid IS NOT NULL
        """, (current_user,))
        total_spent_result = cursor.fetchone()
        total_spent = total_spent_result[0] if total_spent_result else 0

        stats_html = f"""
        <div class="blue-card">
            <h3>📊 Statistik Pembelian</h3>
            <div style="display: flex; justify-content: space-around; background: linear-gradient(135deg, #e8f5e8 0%, #c8e6c9 100%); padding: 20px; border-radius: 10px; border: 2px solid #4caf50;">
                <div style="text-align: center; color: #2e7d32;">
                    <div style="font-size: 28px; font-weight: bold; color: #1b5e20;">{total_orders}</div>
                    <div>📦 Total Order</div>
                </div>
                <div style="text-align: center; color: #2e7d32;">
                    <div style="font-size: 24px; font-weight: bold; color: #1b5e20;">Rp {total_spent*1000:,.0f}</div>
                    <div>💰 Total Belanja</div>
                </div>
            </div>
        </div>
        """
        display(HTML(stats_html))

        # Orders with expandable details
        for order in orders:
            order_id = order[0]
            order_date = order[1]
            order_status = order[2]
            payment_status = order[3] if order[3] else "N/A"
            amount_paid = f"Rp {order[4]*1000:,.0f}" if order[4] else "N/A"

            # Get order items
            cursor.execute("""
                SELECT od.product_id,   p.product_name, od.quantity, p.price, od.price_total
                FROM Order_Details od
                JOIN product p ON od.product_id = p.product_id
                WHERE od.order_id = ?;
            """, (order_id,))
            order_items = cursor.fetchall()

            # Order summary
            order_html = f"""
            <div class="blue-card">
                <h3>📋 Order {order_id}</h3>
                <div style="background: linear-gradient(135deg, #e1f5fe 0%, #b3e5fc 100%); padding: 15px; border-radius: 8px; margin: 10px 0;">
                    <p><strong>Tanggal:</strong> {order_date}</p>
                    <p><strong>Status:</strong> {order_status}</p>
                    <p><strong>Status Pembayaran:</strong> {payment_status}</p>
                    <p><strong>Total:</strong> {amount_paid}</p>
                </div>
            """

            # Order items
            if order_items:
                order_html += '<h4>📦 Item dalam Order:</h4><table style="width:100%; border-collapse: collapse; margin-top: 10px;">'
                order_html += '<tr style="background:#667eea; color:white;"><th style="padding:8px; border:1px solid #ddd;">Produk</th><th style="padding:8px; border:1px solid #ddd;">Jumlah</th><th style="padding:8px; border:1px solid #ddd;">Harga Satuan</th><th style="padding:8px; border:1px solid #ddd;">Total</th></tr>'

                for item in order_items:
                    product_name = item[1]
                    quantity = item[2]
                    unit_price = item[3] * 1000
                    item_total = item[4] * 1000

                    order_html += f'<tr style="background:#f8f9fa;"><td style="padding:6px; border:1px solid #ddd;">{product_name}</td><td style="padding:6px; border:1px solid #ddd; text-align:center;">{quantity}</td><td style="padding:6px; border:1px solid #ddd; text-align:right;">Rp{unit_price:,.0f}</td><td style="padding:6px; border:1px solid #ddd; text-align:right;">Rp{item_total:,.0f}</td></tr>'

                order_html += '</table>'

            order_html += '</div>'
            display(HTML(order_html))
    else:
        no_orders_html = """
        <div class="blue-card">
            <h3>📋 Riwayat Pemesanan</h3>
            <div style="background: linear-gradient(135deg, #fff3e0 0%, #ffe0b2 100%); color: #e65100; text-align: center; padding: 25px; border-radius: 10px; border: 2px solid #ff9800;">
                <div style="font-size: 48px; margin-bottom: 10px;">📭</div>
                <div style="font-size: 16px; font-weight: bold;">Belum ada riwayat pemesanan</div>
                <div style="font-size: 14px; margin-top: 5px;">Mulai berbelanja untuk melihat riwayat order Anda!</div>
            </div>
        </div>
        """
        display(HTML(no_orders_html))

    back_btn = create_button("← Kembali", "secondary")
    back_btn.on_click(lambda b: show_customer_main_menu())

    display(widgets.HBox([back_btn], layout=widgets.Layout(justify_content='center')))

def show_admin_product_search():
    clear_output()
    display(HTML(BLUE_STYLE))

    display(HTML('<div class="blue-header"><h1>📦 Database Produk</h1></div>'))

    search_input = create_input("", "Cari produk...")
    search_btn = create_button("🔍 Cari", "primary")
    back_btn = create_button("← Kembali", "secondary")
    results_output = widgets.Output()

    def on_search_click(b):
        with results_output:
            clear_output()
            if search_input.value:
                products = search_products(search_input.value)
            else:
                products = get_all_products()

            if products:
                df = pd.DataFrame(products, columns=['ID', 'Nama', 'Kategori', 'Harga', 'Deskripsi', 'Stok', 'Rating'])
                display(HTML(f'<div class="blue-card">{df.to_html(index=False)}</div>'))
            else:
                display(HTML('<div class="error-msg">Tidak ada produk ditemukan</div>'))

    def on_back_click(b):
        show_admin_main_menu()

    search_btn.on_click(on_search_click)
    back_btn.on_click(on_back_click)

    controls = widgets.HBox([search_input, search_btn], layout=widgets.Layout(justify_content='center'))
    display(controls)
    display(widgets.HBox([back_btn], layout=widgets.Layout(justify_content='center')))
    display(results_output)

    # Show all products initially
    with results_output:
        products = get_all_products()
        if products:
            df = pd.DataFrame(products, columns=['ID', 'Nama', 'Kategori', 'Harga', 'Deskripsi', 'Stok', 'Rating'])
            display(HTML(f'<div class="blue-card">{df.to_html(index=False)}</div>'))

def show_admin_order_database():
    clear_output()
    display(HTML(BLUE_STYLE))
    display(HTML('<div class="blue-header"><h1>📋 Database Order</h1></div>'))

    # Input search box
    search_input = widgets.Text(
        placeholder='Cari berdasarkan ID Order atau Nama Customer...',
        description='🔍 Cari:',
        layout=widgets.Layout(width='70%')
    )

    search_button = widgets.Button(description='Cari', button_style='info')


    # Container untuk hasil pencarian
    output_area = widgets.Output()

    def fetch_orders(keyword=None):
        query = """
            SELECT o.order_id, c.name_cust, o.order_date, o.status, op.payment_status, op.amount_paid
            FROM Orders o
            JOIN customers c ON o.customer_id = c.customer_id
            JOIN Order_Payments op ON o.order_id = op.order_id
        """
        params = ()
        if keyword:
            query += " WHERE o.order_id LIKE ? OR c.name_cust LIKE ?"
            params = (f'%{keyword}%', f'%{keyword}%')
        query += " ORDER BY o.order_date DESC"

        cursor.execute(query, params)
        return cursor.fetchall()

    def display_orders(orders):
        with output_area:
            output_area.clear_output()
            if orders:
                for order in orders:
                    order_id, customer_name, order_date, order_status, payment_status, amount_paid = order

                    # Get order items
                    cursor.execute("""
                        SELECT od.product_id, p.product_name, od.quantity, p.price, od.price_total
                        FROM order_details od
                        JOIN product p ON od.product_id = p.product_id
                        WHERE od.order_id = ?
                    """, (order_id,))
                    order_items = cursor.fetchall()

                    # Display order details
                    order_html = f"""
                    <div class="blue-card">
                        <h3>📋 Order {order_id}</h3>
                        <div style="background: linear-gradient(135deg, #e1f5fe 0%, #b3e5fc 100%); padding: 15px; border-radius: 8px; margin: 10px 0;">
                            <div style="display: flex; justify-content: space-between; flex-wrap: wrap;">
                                <div>
                                    <p><strong>Customer:</strong> {customer_name}</p>
                                    <p><strong>Tanggal:</strong> {order_date}</p>
                                </div>
                                <div>
                                    <p><strong>Status Order:</strong> {order_status}</p>
                                    <p><strong>Status Bayar:</strong> {payment_status}</p>
                                    <p><strong>Total:</strong> Rp{amount_paid*1000:,.0f}</p>
                                </div>
                            </div>
                        </div>
                    """

                    # Order items table
                    if order_items:
                        order_html += '<h4>📦 Detail Items:</h4><table style="width:100%; border-collapse: collapse; margin-top: 10px;">'
                        order_html += '<tr style="background:#667eea; color:white;"><th style="padding:8px; border:1px solid #ddd;">ID Produk</th><th style="padding:8px; border:1px solid #ddd;">Nama Produk</th><th style="padding:8px; border:1px solid #ddd;">Jumlah</th><th style="padding:8px; border:1px solid #ddd;">Harga Satuan</th><th style="padding:8px; border:1px solid #ddd;">Total</th></tr>'
                        for item in order_items:
                            product_id, product_name, quantity, unit_price, item_total = item
                            unit_price *= 1000
                            item_total *= 1000
                            order_html += f'<tr style="background:#f8f9fa;"><td style="padding:6px; border:1px solid #ddd;">{product_id}</td><td style="padding:6px; border:1px solid #ddd;">{product_name}</td><td style="padding:6px; border:1px solid #ddd; text-align:center;">{quantity}</td><td style="padding:6px; border:1px solid #ddd; text-align:right;">Rp{unit_price:,.0f}</td><td style="padding:6px; border:1px solid #ddd; text-align:right;">Rp{item_total:,.0f}</td></tr>'
                        order_html += '</table>'

                    order_html += '</div>'
                    display(HTML(order_html))
            else:
                display(HTML('<div class="error-msg">❌ Tidak ada order ditemukan</div>'))

    def on_search_clicked(b):
        keyword = search_input.value.strip()
        orders = fetch_orders(keyword)
        display_orders(orders)

    search_button.on_click(on_search_clicked)

    # Load all orders initially
    initial_orders = fetch_orders()
    display(widgets.HBox([search_input, search_button], layout=widgets.Layout(margin='10px 0')))
    display(output_area)
    display_orders(initial_orders)

    # Back button
    back_btn = create_button("← Kembali", "secondary")
    back_btn.on_click(lambda b: show_admin_main_menu())
    display(widgets.HBox([back_btn], layout=widgets.Layout(justify_content='center')))

def show_admin_add_product():
    clear_output()

    # Custom CSS styling
    style = """
    <style>
    .admin-container {
        max-width: 600px;
        margin: 20px auto;
        padding: 25px;
        background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%);
        border-radius: 15px;
        box-shadow: 0 8px 25px rgba(33, 150, 243, 0.2);
        border: 1px solid #2196f3;
    }
    .admin-title {
        color: #1565c0;
        text-align: center;
        margin-bottom: 20px;
        font-family: 'Segoe UI', Arial, sans-serif;
        text-shadow: 0 2px 4px rgba(21, 101, 192, 0.1);
    }
    .input-group {
        margin-bottom: 15px;
    }
    .btn-container {
        text-align: center;
        margin-top: 20px;
    }
    .message-area {
        margin-top: 15px;
        padding: 10px;
        border-radius: 8px;
        text-align: center;
        min-height: 20px;
    }
    </style>
    """

    # Create widgets with blue theme
    title = widgets.HTML(f"{style}<div class='admin-container'><h1 class='admin-title'>➕ Add New Product</h1></div>")

    # Input styling
    input_style = {'description_width': '120px'}
    layout_style = widgets.Layout(width='400px', margin='5px 0')

    name_input = widgets.Text(
        description="📦 Name:",
        placeholder="Enter product name",
        style=input_style,
        layout=layout_style
    )

    price_input = widgets.FloatText(
        description="💰 Price:",
        value=0.0,
        min=0.0,
        style=input_style,
        layout=layout_style
    )

    category_input = widgets.Text(
        description="🏷️ Category:",
        placeholder="Enter category",
        style=input_style,
        layout=layout_style
    )

    stock_input = widgets.IntText(
        description="📊 Stock:",
        value=0,
        min=0,
        style=input_style,
        layout=layout_style
    )

    description_input = widgets.Textarea(
        description="📝 Description:",
        placeholder="Enter product description",
        style=input_style,
        layout=widgets.Layout(width='400px', height='80px', margin='5px 0')
    )

    rating_input = widgets.FloatText(
        description="⭐ Rating:",
        value=0.0,
        min=0.0,
        max=5.0,
        step=0.1,
        style=input_style,
        layout=layout_style
    )

    # Buttons with blue styling
    add_button = widgets.Button(
        description="✅ Add Product",
        button_style='primary',
        layout=widgets.Layout(width='140px', margin='0 10px')
    )

    back_button = widgets.Button(
        description="🔙 Back",
        button_style='info',
        layout=widgets.Layout(width='100px', margin='0 10px')
    )

    message_label = widgets.HTML(value="<div class='message-area'></div>")

    # Event handlers
    def on_add_click(b):
        if not all([name_input.value, price_input.value, category_input.value, stock_input.value, description_input.value]):
            message_label.value = "<div class='message-area' style='background:#ffebee; color:#c62828; border:1px solid #ef5350;'>⚠️ Please fill in all required fields</div>"
            return

        success, msg = add_product(
            name_input.value,
            price_input.value,
            category_input.value,
            stock_input.value,
            description_input.value,
            rating_input.value
        )

        if success:
            message_label.value = f"<div class='message-area' style='background:#e8f5e8; color:#2e7d32; border:1px solid #4caf50;'>✅ {msg}</div>"
            # Clear fields
            name_input.value = ""
            price_input.value = 0.0
            category_input.value = ""
            stock_input.value = 0
            description_input.value = ""
            rating_input.value = 0.0
        else:
            message_label.value = f"<div class='message-area' style='background:#ffebee; color:#c62828; border:1px solid #ef5350;'>❌ {msg}</div>"

    def on_back_click(b):
        show_admin_product_management()

    add_button.on_click(on_add_click)
    back_button.on_click(on_back_click)

    # Display with center alignment
    container = widgets.VBox([
        title,
        widgets.VBox([
            name_input,
            price_input,
            category_input,
            stock_input,
            description_input,
            rating_input,
            widgets.HBox([add_button, back_button], layout=widgets.Layout(justify_content='center')),
            message_label
        ], layout=widgets.Layout(align_items='center'))
    ])

    display(container)

def show_admin_delete_product():
    clear_output()

    # Custom CSS styling
    style = """
    <style>
    .delete-container {
        max-width: 500px;
        margin: 20px auto;
        padding: 25px;
        background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%);
        border-radius: 15px;
        box-shadow: 0 8px 25px rgba(33, 150, 243, 0.2);
        border: 1px solid #2196f3;
    }
    .delete-title {
        color: #1565c0;
        text-align: center;
        margin-bottom: 20px;
        font-family: 'Segoe UI', Arial, sans-serif;
    }
    .warning-box {
        background: linear-gradient(135deg, #fff3e0 0%, #ffe0b2 100%);
        padding: 15px;
        border: 2px solid #ff9800;
        border-radius: 10px;
        margin: 15px 0;
        text-align: center;
        box-shadow: 0 4px 12px rgba(255, 152, 0, 0.2);
    }
    </style>
    """

    # Create widgets
    title = widgets.HTML(f"{style}<div class='delete-container'><h1 class='delete-title'>🗑️ Delete Product</h1></div>")

    # Get products for dropdown
    products = get_all_products()
    product_options = [(f"📦 {p[1]} (ID: {p[0]})", p[0]) for p in products]

    product_dropdown = widgets.Dropdown(
        options=product_options,
        description='🎯 Select:',
        layout=widgets.Layout(width='400px'),
        style={'description_width': '80px'}
    )

    delete_button = widgets.Button(
        description="🗑️ Delete",
        button_style='danger',
        layout=widgets.Layout(width='120px', margin='0 10px')
    )

    back_button = widgets.Button(
        description="🔙 Back",
        button_style='info',
        layout=widgets.Layout(width='100px', margin='0 10px')
    )

    message_label = widgets.HTML(value="")
    confirmation_output = widgets.Output()

    # Event handlers
    def on_delete_click(b):
        if not product_dropdown.value:
            message_label.value = "<div style='background:#ffebee; color:#c62828; padding:10px; border-radius:8px; text-align:center; border:1px solid #ef5350;'>⚠️ Please select a product to delete</div>"
            return

        # Confirmation dialog
        warning_html = widgets.HTML("""
            <div class='warning-box'>
                <h3 style='color:#ef6c00; margin-top:0;'>⚠️ Confirmation Required</h3>
                <p style='margin:10px 0; color:#bf360c;'><strong>Are you sure you want to delete this product?</strong></p>
                <p style='margin:10px 0; color:#bf360c;'>This action cannot be undone!</p>
            </div>
        """)

        confirm_button = widgets.Button(
            description="✅ Confirm",
            button_style='danger',
            layout=widgets.Layout(width='120px', margin='0 10px')
        )

        cancel_button = widgets.Button(
            description="❌ Cancel",
            button_style='warning',
            layout=widgets.Layout(width='100px', margin='0 10px')
        )

        def on_confirm_click(b):
            success, msg = delete_product(product_dropdown.value)

            if success:
                # Refresh dropdown
                products = get_all_products()
                product_options = [(f"📦 {p[1]} (ID: {p[0]})", p[0]) for p in products]
                product_dropdown.options = product_options

                confirmation_output.clear_output()
                message_label.value = f"<div style='background:#e8f5e8; color:#2e7d32; padding:10px; border-radius:8px; text-align:center; border:1px solid #4caf50;'>✅ {msg}</div>"
            else:
                confirmation_output.clear_output()
                message_label.value = f"<div style='background:#ffebee; color:#c62828; padding:10px; border-radius:8px; text-align:center; border:1px solid #ef5350;'>❌ {msg}</div>"

        def on_cancel_click(b):
            confirmation_output.clear_output()
            message_label.value = "<div style='color:#1976d2; text-align:center; padding:10px;'>ℹ️ Delete operation cancelled</div>"

        confirm_button.on_click(on_confirm_click)
        cancel_button.on_click(on_cancel_click)

        confirmation_output.clear_output()
        with confirmation_output:
            display(warning_html)
            display(widgets.HBox([confirm_button, cancel_button], layout=widgets.Layout(justify_content='center')))

    def on_back_click(b):
        show_admin_product_management()

    delete_button.on_click(on_delete_click)
    back_button.on_click(on_back_click)

    # Display with center alignment
    container = widgets.VBox([
        title,
        widgets.VBox([
            product_dropdown,
            widgets.HBox([delete_button, back_button], layout=widgets.Layout(justify_content='center', margin='15px 0')),
            confirmation_output,
            message_label
        ], layout=widgets.Layout(align_items='center'))
    ])

    display(container)

def add_product(product_name, price, category, stock, product_description, rating_product=0):
    try:
        # Generate a new product_id
        def generate_product_id():
            cursor.execute("SELECT product_id FROM product")
            all_ids = cursor.fetchall()

            max_num = 0
            for row in all_ids:
                try:
                    num = int(row[0].replace("PROD", ""))
                    if num > max_num:
                        max_num = num
                except:
                    continue
            return f"PROD{max_num + 1:03d}"

        new_id = generate_product_id()

        # Insert new product
        cursor.execute("""
            INSERT INTO product (product_id, product_name, price, category, stock, product_description, rating_product)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (new_id, product_name, price, category, stock, product_description, rating_product))
        conn.commit()
        return True, f"Product added successfully with ID: {new_id}"
    except Exception as e:
        conn.rollback()
        return False, f"Error adding product: {str(e)}"

def delete_product(product_id):
    try:
        # Check if product exists
        cursor.execute("SELECT * FROM product WHERE product_id = ?", (product_id,))
        product = cursor.fetchone()

        if not product:
            return False, "Product not found"

        # Delete product
        cursor.execute("DELETE FROM product WHERE product_id = ?", (product_id,))
        conn.commit()
        return True, "Product deleted successfully"
    except Exception as e:
        conn.rollback()
        return False, f"Error deleting product: {str(e)}"

# Main function to start the application
def main():
    init_db()
    show_login_page()

# Run the application
if __name__ == "__main__":
    main()

Output()